In [3]:
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.svm import SVC

from bayes_opt import BayesianOptimization
from bayes_opt.util import Colours


def get_data():
    """Synthetic binary classification dataset."""
    data, targets = make_classification(
        n_samples=1000,
        n_features=45,
        n_informative=12,
        n_redundant=7,
        random_state=134985745,
    )
    return data, targets


def svc_cv(C, gamma, degree, data, targets):
    """SVC cross validation.
    This function will instantiate a SVC classifier with parameters C and
    gamma. Combined with data and targets this will in turn be used to perform
    cross validation. The result of cross validation is returned.
    Our goal is to find combinations of C and gamma that maximizes the roc_auc
    metric.
    """
    estimator = SVC(C=C, gamma=gamma, kernel='poly', degree=degree, random_state=2)
    cval = cross_val_score(estimator, data, targets, scoring='roc_auc', cv=4)
    return cval.mean()


def rfc_cv(n_estimators, min_samples_split, max_features, data, targets):
    """Random Forest cross validation.
    This function will instantiate a random forest classifier with parameters
    n_estimators, min_samples_split, and max_features. Combined with data and
    targets this will in turn be used to perform cross validation. The result
    of cross validation is returned.
    Our goal is to find combinations of n_estimators, min_samples_split, and
    max_features that minimzes the log loss.
    """
    estimator = RFC(
        n_estimators=n_estimators,
        min_samples_split=min_samples_split,
        max_features=max_features,
        random_state=2
    )
    cval = cross_val_score(estimator, data, targets, scoring='neg_log_loss', cv=4)
    return cval.mean()


def optimize_svc(data, targets):
    """Apply Bayesian Optimization to SVC parameters."""

    def svc_crossval(expC, expGamma, degree):
        """Wrapper of SVC cross validation.
        Notice how we transform between regular and log scale. While this
        is not technically necessary, it greatly improves the performance
        of the optimizer.
        """
        C = 10 ** expC
        gamma = 10 ** expGamma
        degree = int(degree)
        return svc_cv(C=C, gamma=gamma, degree=degree, data=data, targets=targets)

    optimizer = BayesianOptimization(
        f=svc_crossval,
        pbounds={"expC": (-3, 2), "expGamma": (-4, -1), "degree": (1, 3)},
        random_state=3333,
        verbose=2
    )
    optimizer.maximize(n_iter=50)

    print("Final result:", optimizer.max)


if __name__ == "__main__":
    data, targets = get_data()
    # print(data, targets)
    print(Colours.yellow("--- Optimizing SVM ---"))
    opt = optimize_svc(data, targets)

--- Optimizing SVM ---
|   iter    |  target   |  degree   |   expC    | expGamma  |
-------------------------------------------------------------
|  1        |  0.8392   |  2.503    | -2.454    | -2.542    |
|  2        |  0.8522   |  2.0      |  0.2855   | -3.292    |
|  3        |  0.8893   |  2.226    | -2.402    | -1.863    |
|  4        |  0.8483   |  1.745    | -0.3762   | -3.21     |
|  5        |  0.8392   |  2.243    | -0.7545   | -3.343    |
|  6        |  0.8214   |  1.647    | -2.013    | -2.667    |
|  7        |  0.8904   |  2.212    | -2.41     | -1.845    |
|  8        |  0.9392   |  2.492    | -2.367    | -1.218    |
|  9        |  0.9033   |  3.0      | -1.442    | -1.0      |
|  10       |  0.9168   |  3.0      | -3.0      | -1.0      |
|  11       |  0.8542   |  1.835    | -1.818    | -1.0      |
|  12       |  0.9193   |  3.0      | -2.298    | -1.292    |
|  13       |  0.9387   |  2.194    | -3.0      | -1.0      |
|  14       |  0.9033   |  3.0      |  2.0     